




**Data Parser for grabbing clinical Trial eligibility criteria**

In [ ]:
# CS 333 Final Project Data Parser


import requests
import json

from google.colab import drive
drive.mount('/content/drive')

# api_url = 'https://clinicaltrials.gov/api/query/full_studies?expr=heart+attack&aggFilters=results:with,status:com&min_rnk=1&max_rnk=2&fmt=json'

# single study
# https://clinicaltrials.gov/study/NCT05940077?rank=1&viewType=Table&aggFilters=results:with,status:com&tab=results
api_url = 'https://clinicaltrials.gov/api/v2/studies?aggFilters=results:with,status:com'


params = {
}

response = requests.get(api_url)

if response.status_code == 200:
    print(response.status_code)
else:
    print("Error: ", response.status_code, response.text)

with open("/content/drive/My Drive/outfile_test.json", "w") as outfile:
    raw_json = response.json()
    json_object = json.dumps(raw_json, indent=4)

    # study_data = {}

    # study_data['identificationModule'] = raw_json['studies'][0]['protocolSection']['identificationModule']

    # study_data['eligibilityModule'] = raw_json['studies'][0]['protocolSection']['eligibilityModule']

    # all_data = json.dumps(study_data, indent=4)

    # outfile.write(all_data)

    outfile.write(json_object) # (testing)

    print("Wrote successfully")





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
200
Wrote successfully


In [ ]:
api_url = 'https://clinicaltrials.gov/api/v2/studies?aggFilters=results:with,status:com'

params = {
}

response = requests.get(api_url)

if response.status_code == 200:
    print(response.status_code)
else:
    print("Error: ", response.status_code, response.text)

with open("/content/drive/My Drive/outfile.json", "w") as outfile:
    raw_json = response.json()
    json_object = json.dumps(raw_json, indent=4)

    study_keys = {'studies': []}

    for study in raw_json['studies']:
      study_data = {}

      study_data['identificationModule'] = study['protocolSection']['identificationModule']

      study_data['eligibilityModule'] = study['protocolSection']['eligibilityModule']

      study_keys['studies'].append(study_data)

    outfile.write(json.dumps(study_keys, indent=4))

    # outfile.write(json_object) # (testing)

    print("Wrote successfully")

200
Wrote successfully


**Code for performing named entity recognition on very simple labeled patient health records**

In [3]:
import json
import spacy

from google.colab import drive
drive.mount('/content/drive')

with open("/content/drive/My Drive/data.json", "r") as outfile:
  data = json.load(outfile)


Mounted at /content/drive


In [4]:
!python -m spacy download en_core_web_lg

2023-11-15 22:56:45.932716: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 22:56:45.932804: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 22:56:45.932850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 22:56:47.453907: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 700.3 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
data["data"][0]['entities']

[{'text': "Patient is any gender and any ethnicity and lives in Mississippi; patient displays symptoms of Alzheimer's; patient is 10 years old.",
  'indexes': '119,121',
  'label': 'AGE'},
 {'text': "Patient is any gender and any ethnicity and lives in Mississippi; patient displays symptoms of Alzheimer's; patient is 10 years old.",
  'indexes': '53,64',
  'label': 'LOCATION'},
 {'text': "Patient is any gender and any ethnicity and lives in Mississippi; patient displays symptoms of Alzheimer's; patient is 10 years old.",
  'indexes': '95,106',
  'label': 'DISEASE'}]

In [ ]:
data["data"][0]['entities'][0]['text']

'THIS PATIENT MUST BE 91 YEARS OLD. THEY MUST LIVE/RESIDE/BE IN PROXIMITY OF Texas. HAS COVID DISEASE.'

In [17]:
from spacy.tokens import DocBin

# nlp = spacy.blank("en")

nlp = spacy.load("en_core_web_lg")

doc_bin = DocBin()

In [18]:
from spacy.util import filter_spans

training_data = []

for example in data["data"]:
  temp_lst = []

  text = example['entities'][0]['text']

  labels = []
  for entity in example['entities']:
    start = entity['indexes'].split(',')[0]
    end = entity['indexes'].split(',')[1]
    label = entity['label']
    labels.append((start, end, label))

  doc = nlp.make_doc(text)
  ents = []

  for start, end, label in labels:
    span = doc.char_span(int(start), int(end), label=label, alignment_mode="contract")
    if span is None:
      print("Skipping entity")
    else:
      ents.append(span)
  filtered_ents = filter_spans(ents)
  doc.ents = filtered_ents
  doc_bin.add(doc)

doc_bin.to_disk("train.spacy")


In [14]:
!python -m spacy init fill-config /content/drive/My\ Drive/base_config.cfg config.cfg

2023-11-14 16:35:56.238780: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 16:35:56.238841: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 16:35:56.238869: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 16:35:57.143926: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
!python -m spacy init fill-config --help

2023-11-14 16:30:27.061787: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 16:30:27.061856: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 16:30:27.061891: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 16:30:28.220072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
                                                                                                    
 Usage: python -m spacy init fill-config [OPTIONS] BASE_PATH [OUTPUT_FILE]                          
                                    

In [19]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

2023-11-14 16:37:45.616121: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 16:37:45.616189: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 16:37:45.616269: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 16:37:46.616158: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: [

In [2]:
nlp_ner = spacy.load("model-best")

NameError: ignored

In [1]:
doc = nlp_ner("We will lay out the inclusion/exclusion criteria for this CT. The CT requires patients to be of at least 18 years of age. Patients must live at or reside in Mexico. They should exhibit symptoms of Shnider's and this must be certified by an MD.")

colors = {"LOCATION": "#F67DE3", "AGE": "#7DF6D9", "DISEASE":"#a6e22d"}
options = {"colors": colors}
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

print(doc.ents)



NameError: ignored